In [ ]:
# Standard Libraries of Python
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

# Libraries made for this Proyect
from database.clean_database import database
from data_analisys import data_functions

In [ ]:
class Analisis:
    def __init__(self):
        self.df = pd.read_parquet('database/db.parquet')
        self.lottery = self.df.copy()
        self.lottery['Dates'] = self.lottery['Dates'].dt.year
        self.lottery = self.lottery.drop(columns=['Sorteo','Star_1','Star_2'])
    
    def __transformation_into_columns(self,row):
        for draw in range(1,6):
            if not np.isnan(self.new_lottery.loc[row.Dates,row[f'Nro{draw}']]):
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] += 1
            else:
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] = 1
    
    def __numbers_boolean(self):
        self.skips_winners_bool = pd.DataFrame(False, columns=[int(i) for i in range(1,51)],
            index=range(len(self.df)))
        row_0 = pd.DataFrame(columns=[str(i) for i in range(1, 51)],
            index=[0]).fillna(True)
        for e in range(1,6):
            col_name = f"Nro{e}"
            self.skips_winners_bool = self.skips_winners_bool | (self.df[col_name].to_numpy()[:, None] == range(1,51))
        self.boolean_df = pd.concat([row_0, self.skips_winners_bool]).reset_index(drop=True)

    def __total_average_hits(self,is_star=False,aprox=False):
        divide = 2 if is_star else 5
        self.average = self.hits.apply(lambda hits: hits / len(self.df) / divide).iloc[0].to_frame()
        if aprox:
            return Decimal(self.average.sum().iloc[0]) / Decimal(int(50)) + Decimal(0.001)
        else:
            return Decimal(self.average.sum().iloc[0]) / Decimal(int(50))

    def __natural_rotation(self,index_start,index_end,is_star=False,aprox=False):
        rotation = pd.DataFrame({'hits': self.hits.iloc[0],
            'average_numbers': self.numbers_average.iloc[0],
            'average': self.__total_average_hits(is_star=is_star,aprox=aprox),
            'hits_needed': self.m_hits(is_star=is_star,aprox=aprox)},
            index=range(index_start,index_end+1))
        rotation['difference'] = rotation['hits'] - rotation['hits_needed']
        return rotation
        
    def apply_transformation(self):
        self.new_lottery = pd.DataFrame(columns=[i for i in range(1,51)],
            index=np.arange(self.lottery['Dates'].iloc[0],self.lottery['Dates'].iloc[-1]+1))
        self.lottery.apply(self.__transformation_into_columns,axis=1)
        self.new_lottery.fillna(0,inplace=True)

        self.hits = self.new_lottery.sum().to_frame().rename(columns={0: 'hits'}).T.astype('int32')
        self.mean = self.new_lottery.mean().to_frame().rename(columns={0: 'mean'}).T.astype('float32')
        self.median = self.new_lottery.median().to_frame().rename(columns={0: 'median'}).T.astype('int32')

    def average_hits(self,is_star=False):
        divide = 5 if not is_star else 2
        average = round(self.hits.iloc[0][self.new_lottery.columns] / len(self.lottery) / divide, 6)
        self.numbers_average = pd.DataFrame(columns = self.new_lottery.columns, 
            data = [average], 
            index = ['numbers/stars'])

    def m_hits(self,is_star=False,aprox=False):
        min_hits = self.__total_average_hits(is_star=is_star,aprox=aprox)
        return min_hits * Decimal(int(self.hits.iloc[0,0])) / Decimal(float(self.numbers_average.iloc[0,0]))

    def get_natural_rotations(self,is_star=False):
        self.exact_rotation = self.__natural_rotation(1,50,is_star=is_star,aprox=False)
        self.aprox_rotation = self.__natural_rotation(1,50,is_star=is_star,aprox=True)
    
    def count_skips(self):
        self.__numbers_boolean()
        draws = range(1,len(self.boolean_df))
        dicts = {draw: {key:[] for key in range(1,51)} for draw in draws}
        for e in draws:
            df = self.boolean_df.loc[:e]
            counts = {key: 0 for key in range(1,51)}
            for columns in df:
                counter = 0
                for i in reversed(df[columns]):
                    if not i:
                        counter += 1
                    else:
                        counts[columns] = counter
                        break
            dicts[e].update(counts)
        self.counts = pd.DataFrame.from_dict(dicts,orient='index')

In [ ]:
euromillones = Analisis()
euromillones.apply_transformation()
euromillones.average_hits()
euromillones.get_natural_rotations()

In [ ]:
# Create a template DataFrame with all values set to False
skip_winners_bool = pd.DataFrame(False, columns=[str(i) for i in range(1,51)], index=range(len(euromillones.df)))

# Fill in the True values
euromillones.numbers_boolean(skip_winners_bool)

In [ ]:
euromillones.count_skips()